In [3]:
import time
import datetime
import board

In [7]:
# Adafruit DHT library (Temperature/Humidity)
import adafruit_dht
DHT22Sensor = adafruit_dht.DHT22(board.D16)

Unable to set line 16 to input


In [3]:
# BMP library (Pressure/Temperature)
import adafruit_bmp280
i2c = board.I2C()
bmp280Sensor = adafruit_bmp280.Adafruit_BMP280_I2C(i2c, address = 0x76)
bmp280Sensor.sea_level_pressure = 1013.25

In [4]:
# LEDs
from gpiozero import LED

ledRed = LED(13)
ledYlw = LED(19)
ledGrn = LED(26)

ledRed.off()
ledYlw.off()
ledGrn.off()

# Push-Button
from gpiozero import Button
button = Button(20)

### Getting GPIOs Status

In [5]:
# Get GPIO status data 
def getGpioStatus():
    global timeString
    global buttonSts
    global ledRedSts
    global ledYlwSts
    global ledGrnSts

    # Get time of reading
    now = datetime.datetime.now()
    timeString = now.strftime("%Y-%m-%d %H:%M")
    
    # Read GPIO Status
    buttonSts = button.is_pressed
    ledRedSts = ledRed.is_lit
    ledYlwSts = ledYlw.is_lit
    ledGrnSts = ledGrn.is_lit  

In [6]:
# Print GPIO status data 
def PrintGpioStatus():
    print ("Local Station Time: ", timeString)
    print ("Led Red Status:     ", ledRedSts)
    print ("Led Yellow Status:  ", ledYlwSts)
    print ("Led Green Status:   ", ledGrnSts)
    print ("Push-Button Status: ", buttonSts)

In [7]:
ledRed.on()
ledYlw.on()
ledGrn.on()

In [9]:
getGpioStatus()
PrintGpioStatus()

Local Station Time:  2025-02-07 14:34
Led Red Status:      True
Led Yellow Status:   True
Led Green Status:    True
Push-Button Status:  True


In [10]:
ledRed.off()
ledYlw.off()
ledGrn.off()

In [11]:
getGpioStatus()
PrintGpioStatus()

Local Station Time:  2025-02-07 14:34
Led Red Status:      False
Led Yellow Status:   False
Led Green Status:    False
Push-Button Status:  False


In [12]:
# Acting on GPIOs and printing Status
def controlLeds(r, y, g):
    if (r):
        ledRed.on()
    else:
        ledRed.off()        
    if (y):
        ledYlw.on()
    else:
        ledYlw.off() 
    if (g):
        ledGrn.on()
    else:
        ledGrn.off() 
    
    getGpioStatus()
    PrintGpioStatus()

In [17]:
controlLeds(0, 0, 0)

Local Station Time:  2025-02-07 14:35
Led Red Status:      False
Led Yellow Status:   False
Led Green Status:    False
Push-Button Status:  False


## Getting and displaying Sensor Data

In [18]:
# Read data from BMP280
def bmp280GetData(real_altitude):
    
    temp = bmp280Sensor.temperature
    pres = bmp280Sensor.pressure
    alt =  bmp280Sensor.altitude
    presSeaLevel = pres / pow(1.0 - real_altitude/44330.0, 5.255) 
    
    temp = round (temp, 1)
    pres = round (pres, 2) # absolute pressure in mbar
    alt = round (alt)
    presSeaLevel = round (presSeaLevel, 2) # absolute pressure in mbar
    
    return temp, pres, alt, presSeaLevel

In [19]:
bmp280GetData(960)

(29.7, 907.54, 920, 1018.2)

In [20]:
# Get data (from local sensors)
def getSensorData(altReal=0):
    global timeString
    global humExt
    global tempLab
    global tempExt
    global presSL
    global altLab
    global presAbs
    global buttonSts
	
    # Get time of reading
    now = datetime.datetime.now()
    timeString = now.strftime("%Y-%m-%d %H:%M")
    
    tempLab, presAbs, altLab, presSL = bmp280GetData(altReal) 
    
    tempDHT =  DHT22Sensor.temperature
    humDHT =  DHT22Sensor.humidity
	
    if humDHT is not None and tempDHT is not None:
        tempExt = round (tempDHT)
        humExt = round (humDHT)

In [21]:
DHT22Sensor.temperature, DHT22Sensor.humidity

(29.1, 36.5)

In [22]:
# Display important data on-screen
def printData():
    print ("Local Station Time:             ", timeString)
    print ("External Air Temperature (DHT): ", tempExt, "oC")
    print ("External Air Humidity    (DHT): ", humExt, "%")
    print ("Station Air Temperature  (BMP): ", tempLab, "oC")
    print ("Sea Level Air Pressure:         ", presSL, "mBar")
    print ("Absolute Station Air Pressure:  ", presAbs, "mBar")
    print ("Station Measured Altitude:      ", altLab, "m")

In [23]:
real_altitude = 960 # real altitude of where the BMP280 is installed
getSensorData(real_altitude)
printData()

Local Station Time:              2025-02-07 14:37
External Air Temperature (DHT):  30 oC
External Air Humidity    (DHT):  36 %
Station Air Temperature  (BMP):  29.6 oC
Sea Level Air Pressure:          1018.15 mBar
Absolute Station Air Pressure:   907.5 mBar
Station Measured Altitude:       920 m


## Widgets

In [24]:
# widget library
from ipywidgets import interactive
import ipywidgets as widgets
from IPython.display import display

In [25]:
f = interactive(controlLeds, r=(0,1,1), y=(0,1,1), g=(0,1,1))
display(f)

interactive(children=(IntSlider(value=0, description='r', max=1), IntSlider(value=0, description='y', max=1), …